In [1]:
import pandas as pd

# Fetching Data from SQL server

In [2]:
import pyodbc
import pandas as pd
conn=pyodbc.connect('Driver={SQL Server};'
                    'Server=****************;'
                    'Database=Linkedin_Database;'
                    'Trusted_Connection=yes;')
query = f"select * from [Linkedin_Database].[dbo].[UserProfileData]"
pdf = pd.read_sql(query, conn)
from pyspark.sql import SparkSession
 
 

C:\Users\savit\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:758: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [3]:
df=pd.DataFrame(pdf) 
df.head()

,userId,company,jobTitle,jobLocation,jobDateRange,allSkills
0,596588164,IMS People,Senior Recruitment Consultant,None,Jan 2019 - Present Â· 3 yrs 3 mos,None
1,758396549,IMS People Possible,Senior Recruiter,"Ahmedabad, Gujarat, India",Jan 2021 - Present Â· 1 yr 3 mos,None
2,831767842,Orphosis People Solutions,Recruitment Specialist,None,None,None
3,369878194,Technicolor,Senior Advisor People Partner,"Bangalore Urban, Karnataka, India",Nov 2021 - Present Â· 5 mos,"Management, Recruiting, Leadership, Employee R..."
4,213951696,IMS People,Senior Recruitment Consultant,None,May 2018 - Present Â· 3 yrs 11 mos,"Team Management, Quality Assurance, BPO, Manua..."


Below are the use cases we thought of working on.

We should handle , so that it wont be problem in Azure also also location go with only city (Ahmedabad and we can remove Gujarat and India if we have have)
we need to convert yr - yrs
and mo-mos
1. we can find the good company based on employee worked for long time --- JobDateRange - Months - We can get output
2. We can find the most learned skill by an employee. Using this student can Focus on those skill - All skills unique userId word count
3. For particular job role- which skills are used most - group by jobTitle - word count of all skills - JobTitle we need to do data cleaning (Exa: Sr. as Senior, HR as Human Resource. Ignoring after :,- etc. After / we are appending it to next row - Coding)

Dataset 2 :
4. Which job role will be having highest demand - JobIndustry, JobFunction and jobTitle (Where NULL is there in JobIndustry and Jobfunction will replace it with All)

5. Finding which job category has more number of jobs - JobTitle, JobLocation, groupBy JobIndustry - count of jobTitle
6. Any insights and trying to find patterns in the data - Later we will discuss th

# 1. Most Learned Skill By The Employee

In [4]:
import re
df['allSkills'] = df['allSkills'].str.lower()
df1 = df
df1 = df.drop_duplicates(subset = ["userId"])
df1['allSkills'] = df.allSkills.fillna('Any')
jobSkillList=df1["allSkills"].tolist()
for i in range(len(jobSkillList)):
    line=jobSkillList[i]
    line=line.lower()
    line=re.sub(r'\s+',' ',line)
    line= re.sub('[()].*','',line) 
    #line=re.sub('â€','',line)
    #jobSkillList[i]=re.sub(r'\bSr\b',r'Senior',line)
df1["allSkills"]=jobSkillList
df_skills = df1.allSkills.str.split(',',expand=True).stack().value_counts().reset_index() 
df_skills.columns = ['SkillName', 'Count'] 

C:\Users\savit\AppData\Local\Temp/ipykernel_29560/1424779664.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['allSkills'] = df.allSkills.fillna('Any')
C:\Users\savit\AppData\Local\Temp/ipykernel_29560/1424779664.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1["allSkills"]=jobSkillList


In [5]:
cursor = conn.cursor() 
cursor.execute('''drop table if exists SkillCount''')
cursor.execute('''
		CREATE TABLE SkillCount (
			SkillName varchar(MAX),
			Count smallint)
 
               ''');
conn.commit()

In [6]:
#save the data back to database
# Insert Dataframe into SQL Server:
for index, row in df_skills.iterrows():
     cursor.execute("INSERT INTO SkillCount (SkillName,Count) values(?,?)", row.SkillName, row.Count)
conn.commit()
cursor.close()

In [7]:
query = f"select * from [Linkedin_Database].[dbo].[SkillCount]"
pdf = pd.read_sql(query, conn)
pdf

C:\Users\savit\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:758: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,SkillName,Count
0,management,1407
1,team management,1304
2,recruiting,1238
3,human resources,1115
4,leadership,972
...,...,...
5014,team worker,1
5015,strategy development,1
5016,alm,1
5017,pmo,1


# 2. find the good company based on employee worked for long time

In [8]:
# To convert experience in Months
df['jobDateRange'] = df.jobDateRange.fillna('0')
#df = df[df['jobDateRange'].notna()]
job_range_text=df['jobDateRange'].tolist()
import re
for i in range(len(job_range_text)):
    job_range_text[i]=re.sub(r'\bmo\b',r'mos',job_range_text[i])
    job_range_text[i]=re.sub(r'\byr\b',r'yrs',job_range_text[i])
df["jobDateRange"]=job_range_text
import re
no_month_exp=[]
for i in range(len(job_range_text)):
     matches_year=0
     matches_mon=0
     no_years_in_mon=0
     total_mon=0
     text=job_range_text[i]
     pattern_mon='(\d+) mo'
     matches_mon=re.findall(pattern_mon,text)
     pattern_year='(\d+) yr'
     matches_year=re.findall(pattern_year,text)
     if len(matches_year)==1:
        no_years_in_mon=int(matches_year[0])*12
     if len(matches_mon)!=1:
        total_mon=no_years_in_mon
     else:
        total_mon=no_years_in_mon+int(matches_mon[0]) 
     no_month_exp.append(total_mon)
df["No_month_Exp"]=no_month_exp   

In [9]:
df_exp=df[["company","jobTitle","No_month_Exp","jobLocation"]]

In [10]:
cursor = conn.cursor() 
cursor.execute('''drop table Company_exp''')
cursor.execute('''
		CREATE TABLE Company_exp (
			company varchar(MAX),
			jobTitle varchar(MAX),
            No_month_Exp smallint,
            jobLocation varchar(MAX))
 
               ''');

conn.commit()
#save the data back to database
# Insert Dataframe into SQL Server:
for index, row in df_exp.iterrows():
     cursor.execute("INSERT INTO Company_exp (company,jobTitle,No_month_Exp,jobLocation) values(?,?,?,?)", row.company,row.jobTitle,row.No_month_Exp, row.jobLocation)
conn.commit()
cursor.close()
query = f"select * from [Linkedin_Database].[dbo].[Company_exp]"
pdf = pd.read_sql(query, conn)
pdf

C:\Users\savit\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:758: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,company,jobTitle,No_month_Exp,jobLocation
0,IMS People,Senior Recruitment Consultant,39,None
1,IMS People Possible,Senior Recruiter,15,"Ahmedabad, Gujarat, India"
2,Orphosis People Solutions,Recruitment Specialist,0,None
3,Technicolor,Senior Advisor People Partner,5,"Bangalore Urban, Karnataka, India"
4,IMS People,Senior Recruitment Consultant,47,None
...,...,...,...,...
9934,Bharti Infratel Limited,Chief HR Officer,83,"Gurugram, Haryana, India"
9935,Mercer Consulting,HR Leader,26,None
9936,Zomato,Senior Human Resources Manager,35,None
9937,Publicis Sapient,Senior Manager People Strategy,63,"Gurgaon, India"


# 3.For particular job role- which skills are used most

Data Cleaning.

1. Converting the short forms.  Like Sr- Senior, HR- Human Resource,Jr-Junior, Asst-assistant, Ass- assistant. Leader- Lead
2. 
1) converting short forms- Like Sr- Senior, HR- Human Resource,Jr-Junior, Asst-assistant, Ass- assistant. Leader- Lead
2) Head-, Manager-,Assistant Manager-,Lead-, Team Lead-, Executive-, Talent aquisation-, Senior-, operations-, DM-, Associate  Executive-, (this i will tell about this in meeting)
3)Remove consultant, Numbers , Roman numbers
4) At last we shd apply this- remove the words after :  , ( / |
5) remove dot.

postfix- Manager,head,Team Leader,Lead, Executive, operations
Prefix- Associate, Talent Acquisition,Senior,Associate  Executive

## 3.1 Converting the short forms to abbrevation

In [11]:
#Converting the short forms to abbrevation
import re
df["jobTitle"] = df.jobTitle.fillna('NotFound')
jobTitleList=df["jobTitle"].tolist()
for i in range(len(jobTitleList)):
    line=jobTitleList[i]
    line=re.sub(r'\bJr\b',r'Junior',line)
    line=re.sub(r'\bQA\b',r'Quality Analyst',line)
    line=re.sub(r'\bHR\b',r'Human Resources',line)
    line=re.sub(r'\bAsst\b',r'Assistant Manager',line)
    line=re.sub(r'\bAss\b',r'Assistant Manager',line)
    line=re.sub(r'\bLeader\b',r'Lead',line)
    line=re.sub(r'\bSr\b',r'Senior',line)
    line=re.sub(r'\bSr\b',r'Senior',line)
    
    jobTitleList[i]=re.sub(r'\bSr\b',r'Senior',line)
    
df["jobTitle"]=jobTitleList   

### 3.2 Analyzed to know whether keyword need to be Prefixed or Postfixed by checking the count of keyword in both way.
    Like checking Manager is used more in postfix or prefix 

In [12]:
#To check the count of prefix of keyword and postfix of keyword
df["jobTitle"] = df.jobTitle.fillna('NotFound')
jobTitle_list=df["jobTitle"].tolist()
count_pre=0
count_post=0
for i in range(len(jobTitle_list)):
    line=jobTitle_list[i]
    line=line.lower()
     
    search_pattern_pre= re.search(r'^manager\s[a-b]*',line)
    if search_pattern_pre!=None:
        count_pre=count_pre+1
        #print("Pre "+line)
    else:
        search_pattern_post= re.search(r'.*\smanager$',line)
        if search_pattern_post!=None:
            count_post=count_post+1
            #print("Post "+line)
print("count_pre "+str(count_pre))    
print("count_post "+str(count_post))
 

count_pre 195
count_post 1054


### The count of manager are more in the postfix. so we will make all 148 jobtitle with manger as postfix

"""
manager -
count_pre 148
count_post 887

head -
count_pre 139
count_post 41

team leader -
count_pre 10
count_post 7

leader -
count_pre 0
count_post 69

lead -
count_pre 110
count_post 257

executive -
count_pre 64
count_post 538

director -
count_pre 52
count_post 50

hr -
count_pre 464
count_post 149

analyst -
count_pre 8
count_post 112

partner -
count_pre 5
count_post 213

team - 
count_pre 209
count_post 24

recruiter - 
count_pre 8
count_post 625

president - 
count_pre 5
count_post 23

expert - 
count_pre 0
count_post 12

specialist - 
count_pre 8
count_post 479

 
team lead -
count_pre 35
count_post 41
""" 

 

### According to analysis, we will only do postfix for manager,leader, lead, executive,analyst,partner,recruiter,president,expert,specialist,team lead


In [13]:
# postfix keyword- code to handle manager- and Since manager count postfix is more .so we hv made prefix manager to postfix
import re
df["jobTitle"] = df.jobTitle.fillna('NotFound')
jobTitle_list=df["jobTitle"].tolist()
value=" "
for i in range(len(jobTitle_list)):
    line=jobTitle_list[i]
    line=line.lower()
    search_pattern = re.search('manager - |team leader - |leader - |lead - |executive - |analyst - |partner  - |recruiter - |president - |expert - |specialist - |team lead - ',line)
    if search_pattern!=None:
         
        val=search_pattern.start()
        if val==0:
            split_line=line.split(' - ')
            
            jobTitle_list[i]=split_line[1]+' '+split_line[0]
    else:
        search_result=re.search('^manager\s.*|^team leader\s.*|^leader\s.*|^lead\s.*|^executive\s.*|^analyst\s.*|^partner\s.*|^recruiter\s.*|^president\s.*|^expert\s.*|^specialist\s.*|^team lead\s.*].*',line)
         
        if search_result!=None: # MANAGER, LEAD, TALENT
             if search_result.group()=='team leader' or search_result.group()=='team lead':
                 
                split_spaces=line.split(' ')
                value=split_spaces[2:]+ split_spaces[0:2]
                value_res=" "
                value_res.join(value)
                jobTitle_list[i]=value_res
             
             else:
                 
                split_spaces=line.split(' ')
                value=split_spaces[1:]+ split_spaces[0:1]
                 
                 
                value_res=" "
                 
                jobTitle_list[i]=value_res.join(value)
                 
                 
  
         
df["jobTitle"]=jobTitle_list   

### 3.3 we remove the words after : , ( / |  AND remove the extra spaces . - and consultant word, roman letter, numbers

In [14]:
#  remove the words after :  , ( / | and remove .,spaces
import re
df["jobTitle"] = df.jobTitle.fillna('NotFound')
jobTitleList=df["jobTitle"].tolist()
for i in range(len(jobTitleList)):
    line=jobTitleList[i]
     
    
    line=re.sub(r'\bconsultant\b','',line)
    line=re.sub(r'\d','',line)
    line=re.sub(r'\b[III]\b','',line)
    line=line.lower()
    line=re.sub(r'\s+',' ',line)
    line= re.sub('[,?/|():].*','',line)
    line=re.sub('[.]','',line)
     
    
    jobTitleList[i]=re.sub(r'-','',line) 
    
df["jobTitle"]=jobTitleList 
    
    
    

In [15]:
df

,userId,company,jobTitle,jobLocation,jobDateRange,allSkills,No_month_Exp
0,596588164,IMS People,senior recruitment consultant,None,Jan 2019 - Present Â· 3 yrs 3 mos,None,39
1,758396549,IMS People Possible,senior recruiter,"Ahmedabad, Gujarat, India",Jan 2021 - Present Â· 1 yrs 3 mos,None,15
2,831767842,Orphosis People Solutions,recruitment specialist,None,0,None,0
3,369878194,Technicolor,senior advisor people partner,"Bangalore Urban, Karnataka, India",Nov 2021 - Present Â· 5 mos,"management, recruiting, leadership, employee r...",5
4,213951696,IMS People,senior recruitment consultant,None,May 2018 - Present Â· 3 yrs 11 mos,"team management, quality assurance, bpo, manua...",47
...,...,...,...,...,...,...,...
9934,45303343,Bharti Infratel Limited,chief human resources officer,"Gurugram, Haryana, India",Jun 2013 - Apr 2020 Â· 6 yrs 11 mos,"web design, php, javascript, web development, ...",83
9935,20111206,Mercer Consulting,human resources lead,None,May 2011 - Jun 2013 Â· 2 yrs 2 mos,"photoshop, website development, website updati...",26
9936,190000000,Zomato,senior human resources manager,None,Feb 2018 - Dec 2020 Â· 2 yrs 11 mos,"recruiting, human resources, training, employe...",35
9937,25630801,Publicis Sapient,senior manager people strategy,"Gurgaon, India",Jan 2014 - Mar 2019 Â· 5 yrs 3 mos,"teamwork, recruiting, human resources (hr), le...",63


In [16]:
#3rd use case
from collections import Counter
df["allSkills"] = df.allSkills.fillna('NotFound')
df_JS=df[["jobTitle","allSkills"]]
grouped = df_JS.groupby("jobTitle")
joined ={}
 
for name, group in grouped:
     
    str_text=""
    for i in group["allSkills"]:
         
         
        str_text+=i
        str_text=str_text+","  
    #groupCounts = Counter(str_text.split(" ")).items()
    
     
    str_text=str_text.lower()
    str_text=re.sub(r'\s+',' ',str_text)
    
    str_text=re.sub(r'\([^)]*\)', '', str_text)
   
    groupCounts = Counter(str_text.split(",")).items()
    
    
    joined[name]=groupCounts
    
    
jobtitle_skill = pd.DataFrame.from_dict(joined, orient='index')
jobtitle_skill 

,0,1,2,3,4,5,6,7,8,9,...,961,962,963,964,965,966,967,968,969,970
,"(recruiting, 4)","( sourcing, 6)","( strategic sourcing, 1)","( global sourcing, 1)","( strategy, 3)","( negotiation, 3)","( analytical skills, 1)","( communication, 1)","(talent acquisition, 2)","( recruiting, 2)",...,None,None,None,None,None,None,None,None,None,None
am a recruiter,"(recruiting, 2)","( talent acquisition, 2)","( human resources, 2)","( technical recruiting, 2)","( team management, 2)","( sourcing, 2)","( consulting, 2)","( performance management, 2)","( training, 2)","( crm, 2)",...,None,None,None,None,None,None,None,None,None,None
bussieness devloper & help people to build their side income,"(notfound, 1)","(, 1)",None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
client accounts manager,"(business development, 1)","( project management, 1)","( business strategy, 1)","( marketing strategy, 1)","( electrical engineering, 1)","( electronics, 1)","( troubleshooting, 1)","( automation, 1)","( project engineering, 1)","( six sigma, 1)",...,None,None,None,None,None,None,None,None,None,None
compensation,"(onboarding, 1)","( employee relations, 1)","( compensation & benefit, 1)","( bpo, 1)","( hr policies, 1)","( talent management, 1)","( human resources , 1)","(, 1)",None,None,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zonal human resources talent acquisition & employee experience,"(employee relations, 1)","( performance appraisal, 1)","( personnel management, 1)","( exit interviews, 1)","( interviews, 1)","( career management, 1)","(, 1)",None,None,None,...,None,None,None,None,None,None,None,None,None,None
zonal sales manager,"(sales & marketing, 1)","( distribution management, 1)","( team management, 1)","( strategy, 1)","( sales management, 1)","( business development, 1)","( management, 1)","( distribution, 1)","(, 1)",None,...,None,None,None,None,None,None,None,None,None,None
zonal training manager,"(sales effectiveness, 2)","( soft skills, 2)","( motivational speaking, 2)","( pharmaceutical industry, 2)","( pharmaceutical sales, 2)","( pharmaceutics, 2)","( hospital sales, 2)","( performance management, 2)","( dermatology, 2)","( cardiology, 2)",...,None,None,None,None,None,None,None,None,None,None
zone human resources executive,"(training, 1)","( microsoft word, 1)","( microsoft excel, 1)","( management, 1)","( powerpoint, 1)","( public speaking, 1)","( microsoft office, 1)","( customer service, 1)","( teamwork, 1)","( leadership, 1)",...,None,None,None,None,None,None,None,None,None,None
